# Sonification of Bleeding

## First DEMO in OPS (2019-10-14, Sasan, Philipp, Matthias, Mazda) @Balgrist Universitätsklinik
* Evaluating four examples from 9 initial examples
* First: nature_three (water-shake, birds, rain, thunder) for (moving averages 5s, 30s, and 120s), and church bell for volume, one beat per 50ml.
* Second: algomusic_x, choosing the best example of algomusic for delta and volume.

## Imports

In [1]:
# imports 

import serial
import platform
import time
import sys
import os
import copy
import threading
import matplotlib
import ipywidgets
import scipy.interpolate
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow import keras
from tensorflow.contrib.keras import layers
from scipy import signal
from IPython.display import clear_output
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
from os import path

#import pixiedust

## Train Model & Determine Hardwares

In [2]:
# determine hardware and its port number 

try:
    if platform.system() == 'Windows':
        #sobj_spectro = serial.Serial('COM6', 115200)
        sobj_scale = serial.Serial('COM12', 9600)

    elif platform.system() == 'Darwin':
        # Mac serial call goes here - add your COM Port
        #sobj_spectro = serial.Serial('/dev/tty.usbmodem14301', 115200)
        sobj_scale = serial.Serial('/dev/tty.usbserial-1430', 9600)
except Exception as e:
    print('Exception Thrown: ' + str(e), file=sys.stderr)
    print('Please connect both sensors', file=sys.stderr)
    exit()

## Event-based Sonification

In [3]:
# sc3nb 

import sc3nb as scn
import time
sc = scn.startup()

<IPython.core.display.Javascript object>

Starting sclang...
Done.
Registering UDP callback...
Done.
Booting server...
Done.
-> sc3nb started


## Load Buffers and SynthDefs

In [4]:
# Buffers 

buffers = [
sc.Buffer().load_file("samples/water-shake.wav"),
sc.Buffer().load_file("samples/birds.wav"),
sc.Buffer().load_file("samples/rain.wav"),
sc.Buffer().load_file("samples/thunder.wav"),
sc.Buffer().load_file("samples/seaguls.wav"),
sc.Buffer().load_file("samples/bell.wav")
]

water = buffers[0].bufnum
birds = buffers[1].bufnum 
rain = buffers[2].bufnum
thunder = buffers[3].bufnum
seaguls = buffers[4].bufnum
bell = buffers[5].bufnum

bufnums = [water, birds, rain]


/Users/sasan/anaconda3/lib/python3.7/site-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


In [5]:
%%sc // WaveTables
~wt_sig = 10.collect({
    arg i;
    var numSegs = i.linexp(0,9,4,40).round;

    Env(
        [0]++({1.0.rand}.dup(numSegs-1) * [1,-1]).scramble++[0],
        {exprand(1,i.linexp(0,9,1,50))}.dup(numSegs),
        {[\sine,0,exprand(1,20) * [1,-1].choose].wchoose([9-i,3,i].normalizeSum)}.dup(numSegs)
    ).asSignal(1024);
});

~wt_buf = Buffer.allocConsecutive(10, s, 2048, 1, {
    arg buf, index;
    buf.setnMsg(0, ~wt_sig[index].asWavetable);
});


In [6]:
%%sc // SynthDefs 
SynthDef("pb-simple", { 
    arg out=0, bufnum=0, rate=1, pan=0, amp=0.3, loop=1, 
    lgrt=2, lgamp = 0.5, cf=1000, rq=1, mix=0, room=0, damp=0.5; 
    
    var sig;
    
    sig = PlayBuf.ar(2, bufnum, rate.lag(lgrt)*BufRateScale.kr(bufnum), loop: loop, doneAction: 2);
    sig = BPF.ar(sig, cf, rq);
    sig = FreeVerb2.ar(sig, sig, mix, room, damp);
    
    Out.ar(out, Pan2.ar(sig, pan, amp.lag(lgamp)));
}).add;

SynthDef(\bell, {
    |fs=1, t60=1, pitchy=1, amp=0.25, gate=1|
    var sig, exciter;
    //exciter = Impulse.ar(0);
    exciter = WhiteNoise.ar() * EnvGen.ar(Env.perc(0.001, 0.05), gate) * 0.25;
    sig = Klank.ar(
        `[
            [1, 2, 2.803, 3.871, 5.074, 7.81, 10.948, 14.421],   // freqs
            [1, 0.044, 0.891, 0.0891, 0.794, 0.1, 0.281, 0.079], // amplitudes
            [1, 0.205, 1, 0.196, 0.339, 0.047, 0.058, 0.047]*t60     // ring times
        ],
        exciter,
        freqscale:fs*pitchy);
    sig = FreeVerb.ar(sig) * amp;
    DetectSilence.ar(sig, 0.001, 0.5, doneAction:2);
    Out.ar(0, sig!2);
}).add;

SynthDef(\bpfsaw, {
    arg atk=2, sus=0, rel=3, c1=1, c2=(-1),
    freq=500, detune=0.2, pan=0, cfhzmin=0.1, cfhzmax=0.3,
    cfmin=500, cfmax=2000, rqmin=0.1, rqmax=0.2,
    lsf=200, ldb=0, amp=1, out=0;
    
    var sig, env;
    
    env = EnvGen.kr(Env([0,1,1,0],[atk,sus,rel],[c1,0,c2]),doneAction:2);
    
    sig = Saw.ar(freq * {LFNoise1.kr(0.5,detune).midiratio}!2);
    sig = BPF.ar(
        sig,
        {LFNoise1.kr(
            LFNoise1.kr(4).exprange(cfhzmin,cfhzmax)
        ).exprange(cfmin,cfmax)}!2,
        {LFNoise1.kr(0.1).exprange(rqmin,rqmax)}!2
    );
    sig = BLowShelf.ar(sig, lsf, 0.5, ldb);
    sig = Balance2.ar(sig[0], sig[1], pan);
    sig = sig * env * amp;
    
    Out.ar(out, sig);
}).add;

SynthDef(\bpfsine, {
    arg atk=2, sus=0, rel=3, c1=1, c2=(-1),
    freq=500, detune=0.2, pan=0, cfhzmin=0.1, cfhzmax=0.3,
    cfmin=500, cfmax=2000, rqmin=0.1, rqmax=0.2,
    lsf=200, ldb=0, amp=1, out=0;
    var sig, env;
    env = EnvGen.kr(Env([0,1,1,0],[atk,sus,rel],[c1,0,c2]),doneAction:2);
    sig = SinOsc.ar(freq * {LFNoise1.kr(0.5,detune).midiratio}!2);
    sig = BPF.ar(
        sig,
        {LFNoise1.kr(
            LFNoise1.kr(4).exprange(cfhzmin,cfhzmax)
        ).exprange(cfmin,cfmax)}!2,
        {LFNoise1.kr(0.1).exprange(rqmin,rqmax)}!2
    );
    sig = BLowShelf.ar(sig, lsf, 0.5, ldb);
    sig = Balance2.ar(sig[0], sig[1], pan);
    sig = sig * env * amp;
    Out.ar(out, sig);
}).add;

SynthDef(\osc, {
    arg buf=0, freq=200, detune=0.2,
    amp=0.2, pan=0, out=0, rout=0, rsend=(-20),
    atk=0.01, sus=1, rel=0.01, c0=1, c1=(-1);
    var sig, env, detuneCtrl;
    env = EnvGen.ar(
        Env([0,1,1,0],[atk,sus,rel],[c0,0,c1]),
        doneAction:2
    );
    
    detuneCtrl = LFNoise1.kr(0.1!8).bipolar(detune).midiratio;
    sig = Osc.ar(buf, freq * detuneCtrl, {Rand(0,2pi)}!8);

    sig = Splay.ar(sig);
    sig = LeakDC.ar(sig);
    sig = Balance2.ar(sig[0], sig[1], pan, amp);
    sig = sig * env;
    Out.ar(out, sig);
    Out.ar(rout, sig * rsend.dbamp);
}).add;

SynthDef(\reverb, {
    arg in, predelay=0.1, revtime=1.8,
    lpf=4500, mix=0.15, amp=1, out=0;
    
    var dry, wet, temp, sig;
    
    dry = In.ar(in,2);
    temp = In.ar(in,2);
    wet = 0;
    temp = DelayN.ar(temp, 0,2, predelay);
    16.do{
        temp = AllpassN.ar(temp, 0.05, {Rand(0.001,0.05)}!2, revtime);
        temp = LPF.ar(temp, lpf);
        wet = wet + temp;
    };
    
    sig = XFade2.ar(dry, wet, mix*2-1, amp);
    
    Out.ar(out, sig);
}).add;

In [7]:
%%sc // Busses 
~bus = Dictionary.new;
~bus.add(\reverb -> Bus.audio(s,2));

In [8]:
%%sc // Reveb 
s.waitForBoot({
    s.sync;
    ~out = 0;
    ~mainGroup = Group.new;
    ~reverbGroup = Group.after(~mainGroup);
    ~reverbSynth = Synth.new(\reverb, [
            \amp, 1,
            \predelay, 0.8,
            \revtime, 0.5,
            \lpf, 4500,
            \mix, 0.4,
            \in, ~bus[\reverb],
            \out, ~out,
        ], ~reverbGroup
    );
})


In [9]:
%%sc // Pbinds 

~paddur_min = 4.5;
~paddur_max = 5.5;
~factor_volume = 1;
~bufnum = 1;
~detune_factor = 1;

~mdur_min = 0.99;
~mdur_max = 1;
~mfreq = 1;
~mdetune = 0;
~mrq_min = 0.005;
~mrq_max = 0.008;
~mcf = 1;
~matk = 3;
~msus = 1;
~mrel = 5;
~mamp = 0.9;
~mpan_min = 0;
~mpan_max = 0;

~passcf = 1;


e = Dictionary.new;

e.add(\pad_sine_lf -> {
    ~chords = Pbind(
        \instrument, \osc,
        \dur, Pwhite(Pfunc{~paddur_min}, Pfunc{~paddur_max}),
        \midinote, Pxrand([
            [23,35,54,63,64],
            [45,52,54,59,61,64],
            [28,40,47,56,59,63],
            [42,52,57,61,63]
        ], inf),
        \detune, Pexprand(0.05,0.1) * Pfunc{~detune_factor},
        \atk, Pexprand(2,4),
        \sus, 0,
        \rel, Pexprand(4,6),
        \c0, Pexprand(1,2),
        \c1, Pexprand(1,2).neg,
        \pan, Pwhite(-0.4,0.4),
        \amp, Pexprand(0.001,0.002) * Pfunc{~factor_volume},
        \buf, Pfunc{~bufnum},
        \group, ~mainGroup,
        \out, ~bus[\reverb],
    ).play;
    
    ~marimba = Pbind(
        \instrument, \bpfsaw,
        \dur, Pwhite(Pfunc{~mdur_min}, Pfunc{~mdur_max}),
        \freq, Prand([1/2, 2/3, 1], inf) * Pfunc{~mfreq},
        \detune, Pfunc({~mdetune}),
        \rqmin, Pfunc{~mrq_min},
        \rqmax, Pfunc{~mrq_max},
        \cfmin, Prand((Scale.major.degrees+64).midicps,inf) *
        (Prand(([1,2,4]), inf) * round((Pfunc{~mcf}))),
        \cfmax, Pkey(\cfmin) * Pwhite(1.008,1.025),
        \atk, Pfunc{~matk},
        \sus, Pfunc{~msus},
        \rel, Pfunc{~mrel},
        \amp, Pfunc{~mamp},
        \pan, Pwhite(Pfunc{~mpan_min},Pfunc{~mpan_max}),
        \group, ~mainGroup,
        \out, ~bus[\reverb],
    ).collect({ |event|
    ~marimba_oneEvent = event;
}).play;
});

e.add(\passage_one -> {
~passage_one = Pbind(\instrument, \bell,
    \dur, Pwhite(0.1,0.25, 1),
    \fs, Prand((Scale.major.degrees+64).midicps,inf),
    \amp, Pwhite(0.001,0.003),
    \t60, 9,
    \pitchy, 0.5 + Pfunc{~passcf},
    \group, ~mainGroup,
    \out, 0
).play;
});

e.add(\passage_two -> {
~passage_two = Pbind(\instrument, \bell,
    \dur, Pwhite(0.1,0.25, 3),
    \fs, Prand((Scale.major.degrees+64).midicps,inf),
    \amp, Pwhite(0.001,0.003),
    \t60, 9,
    \pitchy, 0.5 + Pfunc{~passcf},
    \group, ~mainGroup,
    \out, 0
).play;
});

e.add(\passage_three -> {
~passage_three = Pbind(\instrument, \bell,
    \dur, Pwhite(0.05,0.15, 5),
    \fs, Prand((Scale.major.degrees+64).midicps,inf),
    \amp, Pwhite(0.001,0.003),
    \t60, 9,
    \pitchy, 0.5 + Pfunc{~passcf},
    \group, ~mainGroup,
    \out, 0
).play;
});

e.add(\stop -> {
    ~marimba.stop;
    ~chords.stop;
});





e.add(\pad_sine_lf_old -> {
    ~chords = Pbind(
        \instrument, \bpfsine,
        \dur, Pwhite(Pfunc{~paddur_min}, Pfunc{~paddur_max}),
        \midinote, Pxrand([
            [23,35,54,63,64],
            [45,52,54,59,61,64],
            [28,40,47,56,59,63],
            [42,52,57,61,63]
        ], inf),
        \detune, Pexprand(0.05,0.2),
        \cfmin, 500,
        \cfmax, 1000,
        \rqmin, Pexprand(0.01,0.02),
        \rqmax, Pexprand(0.2,0.3),
        \atk, Pwhite(2.0,2.5),
        \rel, Pwhite(6.5,10.0),
        \ldb, 6,
        \amp, 0.2,
        \group, ~mainGroup,
        \out, ~bus[\reverb],
    ).play;
    
    ~marimba = Pbind(
        \instrument, \bpfsaw,
        \dur, Pwhite(Pfunc{~mdur_min}, Pfunc{~mdur_max}),
        \freq, Prand([1/2, 2/3, 1], inf) * Pfunc{~mfreq},
        \detune, Pfunc({~mdetune}),
        \rqmin, Pfunc{~mrq_min},
        \rqmax, Pfunc{~mrq_max},
        \cfmin, Prand((Scale.major.degrees+64).midicps,inf) *
        (Prand(([1,2,4]), inf) * round((Pfunc{~mcf}))),
        \cfmax, Pkey(\cfmin) * Pwhite(1.008,1.025),
        \atk, Pfunc{~matk},
        \sus, Pfunc{~msus},
        \rel, Pfunc{~mrel},
        \amp, Pfunc{~mamp},
        \pan, Pwhite(Pfunc{~mpan_min},Pfunc{~mpan_max}),
        \group, ~mainGroup,
        \out, ~bus[\reverb],
    ).collect({ |event|
    ~marimba_oneEvent = event;
}).play;
});


## Thread for Playing Data

In [10]:
class TrainingModel:
    def __init__(self, units=64):
        self.units = units
        # load the training statistics from the last training from log file
        self.train_stats = pd.read_csv('nn_util/train_stats.csv')

    # model definition
    def build_model(self, num_keys):
        keras_model = keras.Sequential([
            layers.Dense(self.units, activation=tf.nn.relu, input_shape=[num_keys]),
            layers.Dense(self.units, activation=tf.nn.relu),
            layers.Dense(self.units, activation=tf.nn.relu),
            layers.Dense(self.units, activation=tf.nn.relu),
            layers.Dense(1)
        ])

        optimizer = tf.keras.optimizers.RMSprop(0.001)

        keras_model.compile(loss='mean_squared_error',
                            optimizer=optimizer,
                            metrics=['mean_absolute_error', 'mean_squared_error'])
        return keras_model

    # normalize data
    def norm(self, x):
        return (x - self.train_stats['mean']) / self.train_stats['std']

In [11]:
training_model = TrainingModel()

In [12]:
class Bloodplayer:
    
    def __init__(self, pulse_time=0.1, verbose=False):
        self.volume_accumulated = 0
        self.lock = threading.RLock()
        self.stopevent = threading.Event()
        self.callback_fn = None
        self.idx = 0
        self.length = 1000
        self.verbose = verbose
        self.pulse_time = pulse_time
        self.rtime = pulse_time
        self.checkpoint_path = "trained_network/cp.ckpt"
        self.num_keys = 18
        self.oneD_regression_only = 1
        self.model = training_model.build_model(self.num_keys)
        self.model.load_weights(self.checkpoint_path)
        self.column_names = ['channel_1', 'channel_2', 'channel_3', 'channel_4', 'channel_5', 'channel_6',
                    'channel_7', 'channel_8', 'channel_9', 'channel_10', 'channel_11', 'channel_12',
                    'channel_13', 'channel_14', 'channel_15', 'channel_16', 'channel_17', 'channel_18',
                    'target']
        self.max_grams = 0
        self.output_volume = 0
        self.d_volume_old = 0
        self.dd_volume = 0
        self.d_grams = 0
        self.water_accumulated = 0
        self.time_old = 0
        self.d_volume_blood_sum = 0
        self.measurements = 0
        self.delta = [0]
        self.volume = [0]
        self.correction_factor_current = 0
        self.vs0 = [None]
        self.vs1 = [None]
        self.vs2 = [None]
        self.xs = [None]
        self.ts = [None]
        self.ys = [None]
        self.deltas = [0]
        
    def callback_fn_default(self, v):
        os.write(1, f"\r                       \r{v}".encode())
        
    def get_correction(self, d_volume): 
        # init
        measurements_np = []
        self.correction_factor_current = 1
        
        return d_volume * self.correction_factor_current, self.correction_factor_current, measurements_np

    def get_correction_spectro(self, d_volume):

        # init
        measurements_np = []

        # read and decode the signal
        sobj_spectro.flushInput()  # flush the buffer
        output = sobj_spectro.readline()
        output = output.decode("utf-8")
        result = [x.strip() for x in output.split(',')]
        measure = result[1::2]
        try:

            # convert to np array
            measurements_np = np.asarray(measure).astype(float)

            if self.oneD_regression_only:
                # we use a second order sum of sine fit
                x = measurements_np[4]
                a1 = 316.6398
                b1 = 4.9845e-04
                c1 = 2.3296
                a2 = 97.1001
                b2 = 0.0019
                c2 = 3.2029
                correction_factor = (a1 * np.sin(b1 * x + c1) + a2 * np.sin(b2 * x + c2)) / 100
            else:
                # normalize the data
                measurements_normalized = nn_util.norm(measurements_np)
                
                # predict blood/water ratio with neural net
                measure_df = pd.DataFrame(measurements_normalized).transpose()
                prediction = model.predict(measure_df).flatten()
                correction_factor = prediction / 100

            if correction_factor > 1:
                correction_factor = 1
            elif correction_factor < 0:
                correction_factor = 0

            self.correction_factor_current = correction_factor

        except Exception as exc:
            # spectrometer overflow indicates water only
            self.correction_factor_current = 0
            print(str(exc))
            print('Spectrometer overflow')

        return d_volume * self.correction_factor_current, self.correction_factor_current, measurements_np

    def procfn(self):
        self.idx = 0
        self.rtime = 0
        while not self.stopevent.wait(0) and self.idx < self.length-1:
            try:
                time_now = time.time()
                time.sleep(0.05)

                # read the weight from Hx711
                sobj_scale.flushInput()
                grams = sobj_scale.readline()
                grams = float(grams.decode("utf-8"))

                if grams > self.max_grams:
                    self.d_grams = grams - self.max_grams
                    self.max_grams = grams
                else:
                    self.d_grams = 0

                # apply correction factor from spectrometer to only get the blood amount and convert to volume
                self.d_volume_blood, pred, measure_np = self.get_correction(self.d_grams)
                self.d_volume_blood /= 1.060

                # accumulate delta until we print it
                self.d_volume_blood_sum += self.d_volume_blood
                
                # compute accumulated blood volume
                self.volume_accumulated += self.d_volume_blood

                # trend of volume change
                self.dd_volume = self.d_volume_blood_sum - self.d_volume_old

                # print with ~1 Hz
                if (time_now - self.time_old) >= 1:

                    # output
                    #print('----------------------------------------------------')
                    #print('Spectrogram output: ' + str(measure_np))
                    #print('Predicted correction factor: ' + str(pred))
                    #print('----------------------------------------------------')
                    #print('Grams: ' + str(int(grams)))
                    print('----------------------------------------------------')
                    print(str(self.idx) + ': ')
                    print("VOLUME: " + str(int(self.volume_accumulated)))
                    # print("Water accumulated: " + str(int(water_accumulated)))
                    print("DELTA: " + str(int(self.d_volume_blood_sum)))

                    self.delta.append(self.d_volume_blood_sum)
                    v = self.delta[-1]
                    self.volume.append(self.volume_accumulated)
                    if len(self.delta) > 150:
                        self.delta.pop(0)
                        self.volume.pop(0)

                    # reset timer
                    self.time_old = time_now

                    # reset helper variables
                    self.d_volume_old = self.d_volume_blood_sum
                    self.d_volume_blood_sum = 0
                    
                    if self.verbose: 
                        os.write(1, f"\r{self.idx}:{self.idx}                   ".encode())
                    if callable(self.callback_fn):
                        self.callback_fn(self)
                    else:
                        self.callback_fn_default(v)
                    self.rtime += self.pulse_time
                    self.idx = int(self.rtime)
                    time.sleep(self.pulse_time-0.05)

            except Exception as e:
                print(str(e))
                print('Could not read sensor output')
                time.sleep(0.5)
                
        print("done.")
            
    def set_callback(self, fn):
        self.callback_fn = fn
        
    def create_thread(self):
        threadname = "BloodPlayer-thread"
        # check first if it already exists
        if threadname in [t.name for t in threading.enumerate()]:
            print("create_thread: thread is already existing, stop first")
        else:
            self.stopevent.clear()
            self.producer = threading.Thread(name=threadname, target=self.procfn, args=[])
            self.producer.start()

    def stop_thread(self):
        self.stopevent.set()

In [13]:
bloodplayer = Bloodplayer()

W1017 14:54:46.671553 4548007360 deprecation.py:506] From /Users/sasan/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


## Plot Data

In [14]:
%matplotlib

Using matplotlib backend: Qt5Agg


In [15]:
# *** MODIFY BY DEMO ***
delta_max = 15
volume_max = 1000
bloodplayer.volume_accumulated = 0
factor_delta = 1
factor_v0 = 1
factor_v1 = 1
factor_v2 = 1

In [16]:
# create figure
def create_figure():
    global fig, ax, mngr, plmarked, prev_idx 
    global legend_nature, legend_algomus, legend_japan
    
    fig, ax = plt.subplots(1) 
    mngr = plt.get_current_fig_manager(); 
    mngr.window.setGeometry(850, 500, 600, 400)
    ax.clear()
    prev_idx = 0
    plt.xlim(-0.1, volume_max)
    plt.ylim(-0.1, delta_max)
    plmarked, = ax.plot([], [], "r-", lw=1)
    plmarked, = ax.plot([], [], "g-", lw=1)
    plmarked, = ax.plot([], [], "y-", lw=1)
    plmarked, = ax.plot([], [], "C9-", lw=1)
    plmarked, = ax.plot([], [], "r-", lw=1)
    ax.set_xlabel('time in seconds', color='grey')
    ax.set_ylabel('signal value', color='grey')
    ax.set_title(r'Sonification for Process Monitoring in Highly Sensitive Situations')
    legend_nature = plt.legend(('moving average 5s', 'moving average 30s', 'moving average 120s', '0.75 threshold'))
    legend_algomus = plt.legend(('delta', 'filtered delta'))
    legend_japan = plt.legend(('filtered delta', 'moving average 30s', 'moving average 120s'))

def update_plot_nature(self, t, volume_val, xs, vs0, vs1, vs2):
    plmarked.set_data([t,t], [-delta_max, delta_max])
    thrsh.set_data([-10,1000], [0.7, 0.7])
    plv0.set_xdata(xs[0:])
    plv0.set_ydata(vs0[0:])
    plv1.set_xdata(xs[0:])
    plv1.set_ydata(vs1[0:])
    plv2.set_xdata(xs[0:])
    plv2.set_ydata(vs2[0:])
    ax.draw_artist(ax.patch)
    ax.draw_artist(thrsh)
    ax.draw_artist(plv0)
    ax.draw_artist(plv1)
    ax.draw_artist(plv2)
    ax.draw_artist(plmarked)
    progress.value = volume_val
    ax.draw_artist(legend_nature)
    fig.canvas.update()

def update_plot_algomus(self, t, ts, xs, ys, deltas, volume_val):
    global prev_idx

    if not (prev_idx == self.idx):
        filter_plot.set_xdata(ts[0:])
        filter_plot.set_ydata(ys[0:])
        prev_idx = self.idx
    progress.value = volume_val
    plmarked.set_data([t,t], [-delta_max, delta_max])
    pldata.set_xdata(xs[0:])
    pldata.set_ydata(deltas)
    ax.draw_artist(ax.patch)
    ax.draw_artist(pldata)
    ax.draw_artist(filter_plot)
    ax.draw_artist(plmarked)
    ax.draw_artist(legend_algomus)
    fig.canvas.update()

## Sonification

In [17]:
# Custom code for sonifications
#%%pixie_debugger
tau = [5, 20, 40]

v0 = [0, 0]
v1 = [0, 0]
v2 = [0, 0]
takt = [0, 0]
amp = [0, 0, 0]
rate = [0, 0, 0]
pan = [0, 0, 0]
cf = [0, 0, 0]
a = []
b = []
zi = []
zl = [[0, 0, 0]]
cfs = [0.01, 0.05, 0.5]
z = [None]*len(cfs)
sr = 1/bloodplayer.pulse_time
node_base_cont = 5000
node_base_event = 5010
node_base_clock = 5050
takt_rate = 100 # one beat per each takt_rate ml
volume_threshold = 250 # ml threshold for volume
amp_level_mv_volume = 0
    
for k, cf in enumerate(cfs):
    bk, ak = signal.butter(1, cf, fs=sr)
    b.append(bk)
    a.append(ak)
    zi.append(signal.lfilter_zi(bk, ak))
    zl.append([0, 0, 0])

def init(bufnums):
    if not bufnums:
        os.write(1, "no buffer needed to initiate for this sonification   ".encode())
    else:
        for i, bufnum in enumerate(bufnums):
            sc.msg("/s_new", ["pb-simple", (node_base_cont + i), 1, 1, "bufnum", bufnum, "rate", 1, "amp", 0])

def tau_zero(self, volume_val):
    # *** water *** tau 0 = 5 seconds
    if len(self.volume) <= tau[0]:
        v0[0] = ((self.volume[-1] - self.volume[0])/tau[0]) * factor_v0
    else:
        v0[0] = ((self.volume[-1] - self.volume[-(tau[0]+1)])/tau[0]) * factor_v0
    amp[0] = scn.linlin(v0[0], 0, 1.5, 0.2, 0.7)
    amp[0] = np.clip(amp[0], 0.2, 0.7)
    sc.msg("/n_set", [node_base_cont, "rate", 1, "amp", amp[0], "lgrt", 3, "lgamp", 1])
    if v0[0] >= 1.5 * factor_v0:
        amp[0] = scn.linlin(v0[0], 1.5, 4, 0.7, 1)
        amp[0] = np.clip(amp[0], 0.7, 1)
        rate[0] = scn.linlin(v0[0], 1.5, 4, 1.5, 4)
        rate[0] = np.clip(rate[0], 1.5, 4)
        sc.msg("/n_set", [node_base_cont, "rate", rate[0]])

def event_off(buf_node, bufnum):
    sc.msg("/s_new", ["pb-simple", buf_node, 1, 1, "bufnum", bufnum, "rate", 0.5, "amp", 0.4, "loop", 1, "lgrt", 2, "lgamp", 2, "cf", 400])
    time.sleep(2)
    sc.msg("/s_new", ["pb-simple", buf_node, 1, 1, "bufnum", bufnum, "rate", 0.3, "amp", 0.2, "loop", 1, "lgrt", 2, "lgamp", 3, "cf", 200])
    time.sleep(3)
    sc.msg("/s_new", ["pb-simple", buf_node, 1, 1, "bufnum", bufnum, "rate", 0.2, "amp", 0.05, "loop", 0, "lgrt", 1, "lgamp", 1, "cf", 100])
    time.sleep(1)
    sc.msg("/n_free", [buf_node])

def clock_event(bufnum, volume, takt, version):  
    if volume < volume_threshold:
        amp_event = 0
    elif volume >= volume_threshold:
        amp_event = 0.1
    for t in range(1, takt+1):
        sc.msg("/s_new", ["pb-simple", (node_base_clock + t), 1, 1, "bufnum", bufnum, "rate", 1, "amp", amp_event, "loop", 0])
        if version == True:
            time.sleep(3/takt)
        else:
            time.sleep(1)

def plot_and_update_nature(self, volume_val, v0, v1, v2):
    self.xs.append(self.idx)
    self.vs0.append(v0[0])
    self.vs1.append(v1[0])
    self.vs2.append(v2[0])
    update_plot_nature(self, self.idx, volume_val, self.xs, self.vs0, self.vs1, self.vs2)        
        
def plot_and_update_algomus(self, filtered_delta, delta):
    self.xs.append(self.idx)
    self.ts.append(self.rtime)
    self.ys.append((filtered_delta))
    self.deltas.append(delta)
    update_plot_algomus(self, self.rtime, self.ts, self.xs, self.ys, self.deltas, self.volume[-1])
                
def sonification_nature(self):
    # tau 0 = 5 seconds
    # *** water *** 
    tau_zero(self, self.volume[-1])

    # tau 1 = 30 seconds
    amp_level_mv_volume = scn.linlin(self.volume[-1], 0, volume_max, 0, 1)
    amp_level_mv_volume = np.clip(amp_level_mv_volume, 0, 1) 
    
    if len(self.volume) <= tau[1]:
        v1[0] = ((self.volume[-1] - self.volume[0])/tau[1]) * factor_v1
    else:
        v1[0] = ((self.volume[-1] - self.volume[-(tau[1]+1)])/tau[1]) * factor_v1
    
    # *** birds ***
    if v1[0] < 0.5 * factor_v1:
        amp[1] = v1[0]/2 * amp_level_mv_volume
        sc.msg("/n_set", [node_base_cont + 1, "rate", 1, "amp", amp[1]])
    if v1[0] >= 0.5 * factor_v1:
        amp[1] = scn.linlin(v1[0], 0.5, 2, 0.25, 1) 
        amp[1] = np.clip(amp[1], 0.25, 1) * amp_level_mv_volume
        rate[1] = scn.linlin(v1[0], 0.5, 2, 1, 2.5)
        rate[1] = np.clip(rate[1], 1, 2.5)
        sc.msg("/n_set", [node_base_cont + 1, "rate", rate[1], "amp", amp[1]])
    
    # *** rain ***
    if v1[0] < 0.75 * factor_v2:
        sc.msg("/n_set", [node_base_cont + 2, "rate", 0.6, "amp", 0.05, "pan", 1])        

    elif v1[0] >= 0.75 * factor_v2:
        amp[2] = scn.linlin(v1[0], 0.25, 0.5, 0.05, 0.3)
        amp[2] = np.clip(amp[2], 0.1, 0.3) * amp_level_mv_volume
        rate[2] = scn.linlin(v1[0], 0.25, 0.5, 0.6, 1.2)
        rate[2] = np.clip(rate[2], 0.6, 1.2)
        pan[2] = scn.linlin(v1[0], 0.25, 0.5, 1, 0)
        pan[2] = np.clip(pan[2], 1, 0)
        sc.msg("/n_set", [node_base_cont + 2, "rate", rate[2], "amp", amp[2], "pan", pan[2]])        

    # tau 2 = 2 minutes
    if len(self.volume) <= tau[2]:
        v2[0] = ((self.volume[-1] - self.volume[0])/tau[2]) * factor_v2
    else:
        v2[0] = ((self.volume[-1] - self.volume[-(tau[2]+1)])/tau[2]) * factor_v2

    # *** thunder *** 
    if self.volume[-1] >= volume_threshold: # ml
        
        if v2[0] >= 0.7 * factor_v1 and v2[1] < 0.7 * factor_v1:
            sc.msg("/s_new", ["pb-simple", node_base_event, 1, 1, "bufnum", thunder, "rate", 0.9, "amp", 0.9, "loop", 1, "lgrt", 2, "lgamp", 2, "cf", 500])
        if v2[0] < 0.7 * factor_v1 and v2[1] > 0.7 * factor_v1:
            event_off(node_base_event, thunder)
        v2[1] = v2[0] 
    
    os.write(1, f"\r{self.idx}, tau0: {float(v0[0]):4.2},  tau1: {float(v1[0]):4.2},  tau2: {float(v2[0]):4.2},   ".encode())
    
    # plot data
    plot_and_update_nature(self, self.volume[-1], v0, v1, v2)
    
    # clock-event for every 100 ml blood loss
    takt[0] = int(self.volume[-1]/takt_rate)
    if takt[0] > 0 and takt[0] != takt[1]:
        clock_event(bell, self.volume[-1], takt[0], True)
    takt[1] = takt[0]
    
def sonification_algomusic_one(self):
    global a, b, z, zi, zl, ts, ys
    global revtime, mix, predelay, amp 
    global paddur_min, paddur_max, mamp 
    global mdur_min, mdur_max, mfreq, mdetune, mrq_min, mrq_max, mcf, matk, msus, mrel, mpan_min, mpan_max 
    global tau, v0, v1, v2, vs0, vs1, vs2, xs, takt 
    global rate, pan, cf, takt_rate, freq, pasamp, passcf, factor_volume, detune_factor, bufnum

    delta_val = self.delta[-1] * factor_delta
    volume_val = self.volume[-1]
    
    # calculate filters
    for k, cf in enumerate(cfs):
        z[k], zi[k] = signal.lfilter(b[k], a[k], np.array([self.delta[-1]]), zi=zi[k])
        zl[k][0] = copy.copy(z[k][0])

    # delta filtered
    if len(zl[1]) > 3:
        if np.argmax(zl[1]) == 1:
            print('zl: ' + str(zl))
            print('ja argmax works')
            if zl[1][0] < 1:
                %sc ~pasamp = 0
                %sc ~passcf = 1
            else:
                pasamp = scn.linlin(zl[1][0], 1, 3, 0, 0.01)
                pasamp = np.clip(pasamp, 0, 0.01)
                passcf = scn.linlin(zl[1][0], 1, 3, 1, 4)
                passcf = np.clip(passcf, 1, 4)

                %sc ~pasamp = ^pasamp
                %sc ~passcf = ^passcf
                %sc e[\passage_two].value;
        else:
            print('zl not ready')

    zl[1][2] = zl[1][1]
    zl[1][1] = zl[1][0]
    
    revtime = scn.linlin(volume_val, 0, volume_max, 1.8, 0.8)
    revtime = np.clip(revtime, 0.8, 1.8)
    mix = scn.linlin(volume_val, 0, volume_max, 0.5, 0.1)
    mix = np.clip(mix, 0.1, 0.5)
    predelay = scn.linlin(volume_val, 0, volume_max, 0.4, 0.1)
    predelay = np.clip(predelay, 0.1, 0.4)
    amp = scn.linlin(volume_val, 0, volume_max, 0.8, 0.4)        
    amp = np.clip(amp, 0.4, 0.8)
    paddur_min = 4.5-(scn.linlin(delta_val,0,delta_max,0,4))    
    paddur_min = np.clip(paddur_min, 0.5, 4.5)
    paddur_max = 5.5-(scn.linlin(delta_val,0,delta_max,0,4))
    paddur_max = np.clip(paddur_max, 1.5, 5.5)
    factor_volume = scn.linlin(volume_val,0,volume_max,1,20)
    factor_volume = np.clip(factor_volume, 1, 20)
    detune_factor = scn.linlin(volume_val,0,volume_max,1,5)
    detune_factor = np.clip(detune_factor, 1, 5)
    bufnum = scn.linlin(volume_val, 0, volume_max,0,9)
    bufnum = np.clip(bufnum, 0, 9)
    mdur_min = scn.linlin(delta_val,0,delta_max,0.99,0.05)
    mdur_min = np.clip(mdur_min, 0.05, 0.99)
    mdur_max = scn.linlin(delta_val,0,delta_max,1,0.1)
    mdur_max = np.clip(mdur_max, 0.1, 1)
    mfreq = scn.linlin(delta_val,0,delta_max,1,4)
    mfreq = np.clip(mfreq, 1, 4)
    mdetune = scn.linlin(delta_val,0,delta_max,0,2)
    mdetune = np.clip(mdetune, 0, 2)
    mrq_min = scn.linlin(volume_val,0,volume_max,0.005,0.09)
    mrq_min = np.clip(mrq_min, 0.005,0.09)
    mrq_max = scn.linlin(volume_val,0,volume_max,0.008,0.2)
    mrq_max = np.clip(mrq_max, 0.008, 0.2)
    mcf = scn.linlin(delta_val,0,delta_max,1,5)
    mcf = np.clip(mcf, 1, 5)
    matk = scn.linlin(volume_val,0,volume_max,3,2)
    matk = np.clip(matk, 2, 3)
    msus = scn.linlin(volume_val,0,volume_max,1,0.5)
    msus = np.clip(msus, 0.5, 1)
    mrel = scn.linlin(volume_val,0,volume_max,5,3)
    mrel = np.clip(mrel, 3, 5)
    mamp = scn.linlin(delta_val,0,delta_max,0.05,0.7)
    mamp = np.clip(mamp, 0.05, 0.7)
    mpan_min = scn.linlin(delta_val,0,delta_max,0,-1)
    mpan_min = np.clip(mpan_min, 0, -1)
    mpan_max = scn.linlin(delta_val,0,delta_max,0,1)
    mpan_max = np.clip(mpan_max, 0, 1)

    %sc ~reverbSynth.set(\revtime, ^revtime, \mix, ^mix, \predelay, ^predelay, \amp, ^amp)        
    %sc ~paddur_min = ^paddur_min
    %sc ~paddur_max = ^paddur_max
    %sc ~factor_volume = ^factor_volume
    %sc ~detune_factor = ^detune_factor
    %sc ~bufnum = ^bufnum.asInteger
    %sc ~mdur_min = ^mdur_min
    %sc ~mdur_max = ^mdur_max
    %sc ~mfreq = ^mfreq
    %sc ~mdetune = ^mdetune
    %sc ~mrq_min = ^mrq_min
    %sc ~mrq_max = ^mrq_max
    %sc ~mcf = ^mcf
    %sc ~matk = ^matk
    %sc ~msus = ^msus
    %sc ~mrel = ^mrel
    %sc ~mamp = ^mamp
    %sc ~mpan_min = ^mpan_min
    %sc ~mpan_max = ^mpan_max
    
    plot_and_update_algomus(self, self.delta[-1], zl[1][0])
    
    # clock-event for every 50 ml blood loss
    takt[0] = int(self.volume[-1]/takt_rate)
    if takt[0] > 0 and takt[0] != takt[1]:
        clock_event(bell, takt[0], True)
    takt[1] = takt[0]
    
def quit():
    sc.msg("/n_free", node_base_cont)
    sc.msg("/n_free", node_base_cont + 1)
    sc.msg("/n_free", node_base_cont + 2)
    sc.msg("/n_free", node_base_event)

## GUI

In [18]:
# GUI

selected_init = 0

# filling blood bar
progress = ipywidgets.FloatProgress(value=5, min=0, max=volume_max, description="volume 0-%s ml" %volume_max, bar_style='danger',layout=Layout(width='20%', height='250px'))
progress.orientation = 'vertical'
display(progress)

def select_sonification(selection):
    global selected_init, thrsh, plv0, plv1, plv2, filter_plot, pldata, amp
    
    bloodplayer.set_callback(selection)
        
    if selection == sonification_nature:
        bloodplayer.pulse_time = 1
        selected_init = 2
        create_figure()
        plv0, = ax.plot([], [], "r-", lw=1)
        plv1, = ax.plot([], [], "g-", lw=1)
        plv2, = ax.plot([], [], "y-", lw=1)
        thrsh, = ax.plot([], [], "C9-", lw=1)
        
    if selection == sonification_algomusic_one:
        bloodplayer.pulse_time = 1
        selected_init = 3
        create_figure()
        filter_plot = plt.plot([], [], 'r-', lw=1)[0]
        pldata, = ax.plot([], [], "g-", lw=0.5)
    
interact(select_sonification, selection = {
    'nature': sonification_nature,
    'algoMusic_one': sonification_algomusic_one,
})

def start(b):
    global bloodplayer
    bloodplayer.create_thread()
    init(bufnums)
    if selected_init == 3:
        %sc e[\pad_sine_lf].value;
    os.write(2, f"event  \r{selected_init}  initiated         ".encode())

b1 = ipywidgets.Button(description='Start') 
b1.on_click(start)

def stop(b):
    global bloodplayer
    print("stop")
    bloodplayer.stop_thread()
    quit()
    if selected_init > 2:
        %sc e[\stop].value;
        
b2 = ipywidgets.Button(description='Stop') 
b2.on_click(stop)

out = ipywidgets.Output()
ipywidgets.HBox([b1, b2, out])

FloatProgress(value=5.0, bar_style='danger', description='volume 0-1000 ml', layout=Layout(height='250px', wid…

interactive(children=(Dropdown(description='selection', options={'nature': <function sonification_nature at 0x…

----------------------------------------------------
0: 
VOLUME: 0
DELTA: 0
----------------------------------------------------
1: 
VOLUME: 0
DELTA: 0
----------------------------------------------------
2: 
VOLUME: 0
DELTA: 0
----------------------------------------------------
3: 
VOLUME: 0
DELTA: 0
----------------------------------------------------
4: 
VOLUME: 0
DELTA: 0
----------------------------------------------------
5: 
VOLUME: 0
DELTA: 0
----------------------------------------------------
6: 
VOLUME: 0
DELTA: 0
----------------------------------------------------
7: 
VOLUME: 0
DELTA: 0
----------------------------------------------------
8: 
VOLUME: 0
DELTA: 0
----------------------------------------------------
9: 
VOLUME: 0
DELTA: 0
----------------------------------------------------
10: 
VOLUME: 83
DELTA: 82
----------------------------------------------------
11: 
VOLUME: 121
DELTA: 38
----------------------------------------------------
12: 
VOLUME: 121
DELTA: 0
--

In [19]:
# adapt to music-scale: event clock to the algomus
# bell in algomus better choice?
# bell in nature better choice?